In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [2]:
df = pd.read_csv('hand_keypoints.csv', header=None)
df[0] = df[0].astype(str)
unique_labels = df[0].unique()


C:\Users\91977\AppData\Local\Temp\ipykernel_13192\1062197483.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('hand_keypoints.csv', header=None)


In [3]:
features = df.iloc[:, 1:]
labels = df[[0]]


In [4]:
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)


C:\Users\91977\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [5]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', mode='min', verbose=1, patience=2)


In [6]:
X_train, X_val, y_train, y_val = train_test_split(
    features, encoded_labels, test_size=0.2, random_state=42)


In [7]:
model = keras.Sequential([
    layers.Dense(1470, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(832, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(428, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(264, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(35, activation='softmax')
])


In [8]:
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)


In [9]:
model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=128,
    validation_split=0.2,
    callbacks=[early_stopping]
)


Epoch 1/50
289/289 ━━━━━━━━━━━━━━━━━━━━ 13s 41ms/step - accuracy: 0.3759 - loss: 2.1186 - val_accuracy: 0.9575 - val_loss: 0.1124
Epoch 2/50
289/289 ━━━━━━━━━━━━━━━━━━━━ 12s 41ms/step - accuracy: 0.9172 - loss: 0.2403 - val_accuracy: 0.9830 - val_loss: 0.0554
Epoch 3/50
289/289 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.9483 - loss: 0.1531 - val_accuracy: 0.9905 - val_loss: 0.0329
Epoch 4/50
289/289 ━━━━━━━━━━━━━━━━━━━━ 11s 40ms/step - accuracy: 0.9643 - loss: 0.1022 - val_accuracy: 0.9925 - val_loss: 0.0325
Epoch 5/50
289/289 ━━━━━━━━━━━━━━━━━━━━ 11s 39ms/step - accuracy: 0.9704 - loss: 0.0892 - val_accuracy: 0.9861 - val_loss: 0.0339
Epoch 6/50
289/289 ━━━━━━━━━━━━━━━━━━━━ 21s 40ms/step - accuracy: 0.9709 - loss: 0.0923 - val_accuracy: 0.9932 - val_loss: 0.0279
Epoch 7/50
289/289 ━━━━━━━━━━━━━━━━━━━━ 12s 40ms/step - accuracy: 0.9783 - loss: 0.0713 - val_accuracy: 0.9868 - val_loss: 0.0312
Epoch 8/50
289/289 ━━━━━━━━━━━━━━━━━━━━ 12s 41ms/step - accuracy: 0.9760 - loss: 0.0659 - 

In [10]:
test_loss, test_accuracy = model.evaluate(X_val, y_val, verbose=0)


In [11]:
predictions = model.predict(X_val)
predicted_classes = np.argmax(predictions, axis=1)


361/361 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step


In [12]:
accuracy = accuracy_score(y_val, predicted_classes)
precision = precision_score(y_val, predicted_classes, average='macro')
recall = recall_score(y_val, predicted_classes, average='macro')
f1 = f1_score(y_val, predicted_classes, average='macro')


In [13]:
print("Test Accuracy:", accuracy)
print("Test Precision:", precision)
print("Test Recall:", recall)
print("Test F1-Score:", f1)


Test Accuracy: 0.9936648442245943
Test Precision: 0.9951280455681625
Test Recall: 0.9919297595231754
Test F1-Score: 0.9932907002271731


In [14]:
model.save('model_v0.h5')
